In [54]:
blueprintdata = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.""".split("\n")

In [1]:
with open("input.txt") as file:
    blueprintdata = file.read().split("\n")

In [2]:
from typing import Dict, List, Tuple
from parse import parse
from nested_lookup import nested_lookup

In [3]:
def update_storage(robots: Dict[str, int], storage: Dict[str, int]) -> Dict[str, int]:
    for robot in robots:
        storage[robot.split("_")[0]] += robots[robot]
    return storage

In [4]:
def buildable_robots(
    robots: Dict[str, int],
    storage: Dict[str, int],
    blueprint: Dict[str, Dict[str, int]],
) -> List[str]:
    buildable_bots = []
    for robot in robots:
        robot_resources = nested_lookup(robot.split("_")[0], blueprint)
        if robot_resources:
            if all(
                [
                    storage[requirement] >= blueprint[robot][requirement]
                    for requirement in blueprint[robot]
                ]
            ) and robots[robot] < max(robot_resources):
                buildable_bots.append(robot)
        else:
            if all(
                [
                    storage[requirement] >= blueprint[robot][requirement]
                    for requirement in blueprint[robot]
                ]
            ):
                buildable_bots.append(robot)
    return buildable_bots


In [5]:
def build_robot(robot_to_be_build: str, robots: Dict[str, int], storage: Dict[str, int], blueprint:Dict[str, Dict[str, int]]) -> Tuple[Dict[str, int],Dict[str, int]]:
    robots[robot_to_be_build] += 1
    costs = blueprint[robot_to_be_build]
    for item in costs:
        storage[item] -= costs[item]
    return robots, storage

In [6]:
def search_best(time_left:int, storage: Dict[str, int], robots: Dict[str, int], blueprint: Dict[str, Dict[str, int]], lowest):
    if time_left == 0:
        return storage["geode"]
    
    if storage["geode"] + time_left * robots["geode_robot"] + time_left * (time_left -1) /2 <= lowest:
        return 0
    
    possible_builds = buildable_robots(robots=robots, storage=storage, blueprint=blueprint)
    
    time_left -= 1
    storage = update_storage(storage=storage, robots=robots)
    
    best = lowest
    
    if "geode_robot" in possible_builds:
        gd_robots, gd_storage = build_robot("geode_robot", robots.copy(), storage.copy(), blueprint)
        best = max(best, search_best(time_left, gd_storage, gd_robots, blueprint, best))
    
    if "obsidian_robot" in possible_builds:
        ob_robots, ob_storage = build_robot("obsidian_robot", robots.copy(), storage.copy(), blueprint)
        best = max(best, search_best(time_left, ob_storage, ob_robots, blueprint, best))
    
    if "clay_robot" in possible_builds:
        cl_robots, cl_storage = build_robot("clay_robot", robots.copy(), storage.copy(), blueprint)
        best = max(best, search_best(time_left, cl_storage, cl_robots, blueprint, best))
    
    if "ore_robot" in possible_builds:
        or_robots, or_storage = build_robot("ore_robot", robots.copy(), storage.copy(), blueprint)
        best = max(best, search_best(time_left, or_storage, or_robots, blueprint, best))
    
    best = max(best, search_best(time_left, storage, robots, blueprint, best))
    return best

In [7]:
blueprints = {}
for blueprint in blueprintdata:
    matches = parse(
        "Blueprint {}: Each ore robot costs {} ore. Each clay robot costs {} ore. Each obsidian robot costs {} ore and {} clay. Each geode robot costs {} ore and {} obsidian.",
        blueprint,
    )
    blueprints[matches[0]] = {
        "ore_robot": {"ore": int(matches[1])},
        "clay_robot": {"ore": int(matches[2])},
        "obsidian_robot": {
            "ore": int(matches[3]),
            "clay": int(matches[4]),
        },
        "geode_robot": {"ore": int(matches[5]), "obsidian": int(matches[6])},
    }


In [8]:
## Way to opportunistic building method
end_result = {}
for blueprint in blueprints:
    robots_to_be_build = []
    storage = {"ore":0, "clay": 0, "obsidian": 0, "geode":0}
    robots = {"ore_robot":1, "clay_robot": 0, "obsidian_robot": 0, "geode_robot":0}
    for time in range(1,25):
        storage = update_storage(robots=robots, storage=storage)
        if robots_to_be_build:
            robots, storage = build_robot(robots_to_be_build[0], robots, storage, blueprints[blueprint])
        print(f"At time {time} there are {robots}, which gives {storage}")
        robots_to_be_build = buildable_robots(robots=robots, storage=storage, blueprint=blueprints[blueprint])
    end_result[blueprint] = {"storage": storage.copy(),
                             "robots": robots.copy()}

At time 1 there are {'ore_robot': 1, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0}, which gives {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
At time 2 there are {'ore_robot': 1, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0}, which gives {'ore': 2, 'clay': 0, 'obsidian': 0, 'geode': 0}
At time 3 there are {'ore_robot': 1, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0}, which gives {'ore': 3, 'clay': 0, 'obsidian': 0, 'geode': 0}
At time 4 there are {'ore_robot': 1, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0}, which gives {'ore': 4, 'clay': 0, 'obsidian': 0, 'geode': 0}
At time 5 there are {'ore_robot': 2, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0}, which gives {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
At time 6 there are {'ore_robot': 2, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0}, which gives {'ore': 3, 'clay': 0, 'obsidian': 0, 'geode': 0}
At time 7 there are {'ore_robot': 2, 'clay_robot': 0, 'obsidian_robot'

In [9]:
quality = 0
for i in end_result:
    quality += int(i) * end_result[i]["storage"]["geode"]
quality # Needs to be larger than 1295

0

In [8]:
# Progress bar on a single-machine scheduler
import dask

In [9]:
from tqdm import tqdm
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=3)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 12,Total memory: 63.93 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62136,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 63.93 GiB
Comm: tcp://127.0.0.1:62152,Total threads: 4
Dashboard: http://127.0.0.1:62153/status,Memory: 21.31 GiB
Nanny: tcp://127.0.0.1:62139,


In [10]:
def run_blueprint(blueprint, max_time=24):
    end_result = {}
    storage = {"ore":0, "clay": 0, "obsidian": 0, "geode":0}
    robots = {"ore_robot":1, "clay_robot": 0, "obsidian_robot": 0, "geode_robot":0}
    end_result[blueprint] = search_best(max_time, storage, robots, blueprints[blueprint], 0)
    return end_result

In [11]:
end_result = {}
lazy_results = []
for blueprint in blueprints:
    lazy_result = dask.delayed(run_blueprint)(blueprint)
    lazy_results.append(lazy_result)

In [13]:
all_results = dask.compute(*lazy_results)

In [14]:
all_results

({'1': 3},
 {'2': 1},
 {'3': 0},
 {'4': 0},
 {'5': 9},
 {'6': 0},
 {'7': 6},
 {'8': 2},
 {'9': 6},
 {'10': 4},
 {'11': 2},
 {'12': 0},
 {'13': 0},
 {'14': 1},
 {'15': 0},
 {'16': 3},
 {'17': 0},
 {'18': 3},
 {'19': 0},
 {'20': 2},
 {'21': 3},
 {'22': 3},
 {'23': 8},
 {'24': 1},
 {'25': 16},
 {'26': 0},
 {'27': 5},
 {'28': 7},
 {'29': 3},
 {'30': 1})

In [17]:
for result in all_results:
    end_result |= result
quality = 0
for i in end_result:
    quality += int(i) * end_result[i]
quality

1565

In [16]:
end_result

{}

In [11]:
import math
end_result = {}
lazy_results = []
for blueprint in ["1","2","3"]:
    lazy_result = dask.delayed(run_blueprint)(blueprint, 32)
    lazy_results.append(lazy_result)
    
all_results = dask.compute(*lazy_results)

elephant_result = {}
for result in all_results:
    elephant_result |= result

math.prod(elephant_result.values())

10672

In [12]:
elephant_result

{'1': 29, '2': 23, '3': 16}